In [ ]:
# 필요한 패키지 설치 (최초 1회만 실행)
# !pip install yfinance ta pandas matplotlib

import pandas as pd
import yfinance as yf
import ta
import numpy as np
import matplotlib.pyplot as plt

# 1. 데이터 수집 (삼성전자 예시)
ticker = "005930.KS"
start_date = "2020-01-01"
end_date = "2024-12-31"

df = yf.download(ticker, start=start_date, end=end_date)

# 2. 기술적 지표 및 조건 계산
df["RSI"] = ta.momentum.RSIIndicator(df["Close"]).rsi()
df["Volume_Spike"] = df["Volume"] > df["Volume"].rolling(5).mean() * 2
df["Bullish_Candle"] = df["Close"] > df["Open"]

# 3. 공매도 관련 지표 (실제 연동 시 별도 크롤링 필요 – 여기선 임의 생성)
np.random.seed(42)
df["공매도_비중"] = np.random.uniform(0.5, 2.5, len(df))  # 상위 5% 비중 설정
df["공매도_급증"] = df["공매도_비중"].diff() > 0.5
df["업틱룰_발동"] = np.random.choice([True, False], size=len(df), p=[0.1, 0.9])
df["외인_순매수"] = np.random.randint(-1000, 1000, size=len(df))
df["기관_순매수"] = np.random.randint(-1000, 1000, size=len(df))

# 4. 매수 시그널 조건 조합
df["Signal"] = (
    (df["공매도_비중"] > 1.5) &
    (df["공매도_급증"]) &
    (df["업틱룰_발동"]) &
    (df["RSI"] < 30) &
    (df["Bullish_Candle"]) &
    (df["Volume_Spike"]) &
    (df["외인_순매수"] > 0) &
    (df["기관_순매수"] > 0)
)

# 5. 백테스트: 매수 다음날 시가 → 5일 후 종가 매도
df = df.reset_index()
trades = []

for i in range(len(df) - 6):
    if df.loc[i, "Signal"]:
        entry_date = df.loc[i + 1, "Date"]
        entry_price = df.loc[i + 1, "Open"]
        exit_price = df.loc[i + 6, "Close"]
        ret = (exit_price - entry_price) / entry_price
        trades.append({
            "Entry Date": entry_date,
            "Entry Price": entry_price,
            "Exit Price": exit_price,
            "Return": ret
        })

results_df = pd.DataFrame(trades)
results_df["Cumulative Return"] = (1 + results_df["Return"]).cumprod()

# 결과 출력
print(results_df)
results_df[["Entry Date", "Return", "Cumulative Return"]].plot(x="Entry Date", marker='o', title="공매도 기반 전략 수익률 추이")
plt.grid(True)
plt.show()
